# Лабораторна робота №2, Кривохата Марія, ІМ-21

In [43]:
import re
import random
import nltk
import pandas as pd
import numpy as np
import string
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize

In [2]:
data_df = pd.read_csv("news2.csv")
data_df.head(10)

,Unnamed: 0,text,label
0,9383,Three former JPMorgan employees who worked in ...,12
1,11899,Wall Street looks set for a modestly softer op...,15
2,6179,CHILE'S CENTRAL BANK SAYS EXPECTS MORE INTERES...,1
3,5888,How is the Fed responding to inflation? @Nasd...,1
4,15391,$VFC - V.F. Corporation: A Potential Trade Opp...,18
5,5364,Crude Oil back under 100 getting smoked,6
6,16185,"$USEA $NOVN $CNK - LGHL, CNK and NOVN are amon...",19
7,7360,While BlackRock sought to blame 2022’s market ...,9
8,11464,The EU stood by its previous guidance that san...,14
9,6537,JUST IN: The Philippine central bank unexpecte...,1


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mkriv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mkriv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [122]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'new'}
    stop_words.update(custom_stopwords)
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

In [123]:
def preprocess_text(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    return text

data_df['clean_text'] = data_df['text'].apply(preprocess_text)
tokenized_docs = [doc.split() for doc in data_df['clean_text']]

In [132]:
data_df.head(10)

,Unnamed: 0,text,label,clean_text
0,9383,Three former JPMorgan employees who worked in ...,12,three former jpmorgan employees worked banks p...
1,11899,Wall Street looks set for a modestly softer op...,15,wall street looks set modestly softer open tra...
2,6179,CHILE'S CENTRAL BANK SAYS EXPECTS MORE INTERES...,1,chiles central bank says expects interest rate...
3,5888,How is the Fed responding to inflation? @Nasd...,1,fed responding inflation nasdaq chief economis...
4,15391,$VFC - V.F. Corporation: A Potential Trade Opp...,18,vfc vf corporation potential trade opportunity...
5,5364,Crude Oil back under 100 getting smoked,6,crude oil back getting smoked
6,16185,"$USEA $NOVN $CNK - LGHL, CNK and NOVN are amon...",19,usea novn cnk lghl cnk novn among ore market g...
7,7360,While BlackRock sought to blame 2022’s market ...,9,blackrock sought blame market carnage theres t...
8,11464,The EU stood by its previous guidance that san...,14,eu stood previous guidance sanctioned goods tr...
9,6537,JUST IN: The Philippine central bank unexpecte...,1,philippine central bank unexpectedly raises ra...


In [124]:
def filter_short_tokens(tokens, min_len=3):
    return [token for token in tokens if len(token) >= min_len]

tokenized_docs = [filter_short_tokens(doc) for doc in tokenized_docs]

In [125]:
bigram = Phrases(tokenized_docs, min_count=20, threshold=20, delimiter='_')
bigram_model = Phraser(bigram)
bigrams_applied = [' '.join(bigram_model[doc]) for doc in tokenized_docs]

### 1.Застосувати приховане семантичне індексування бібліотеки scikit-learn для моделювання тем.

In [126]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.6, min_df=20)
bow_matrix = vectorizer.fit_transform(bigrams_applied)

n_topics = 3
lsi_model = TruncatedSVD(n_components=n_topics, n_iter=10, random_state=42)
doc_topic_matrix = lsi_model.fit_transform(bow_matrix)

In [127]:
dictionary = Dictionary(tokenized_docs)

terms = vectorizer.get_feature_names_out()
topics = []

for topic_weights in lsi_model.components_:
    top_indices = topic_weights.argsort()[::-1][:10]
    top_words = [terms[i] for i in top_indices]
    topics.append(top_words)

coherence_model_cv = CoherenceModel( topics=topics, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
coherence_model_umass = CoherenceModel( topics=topics, texts=tokenized_docs, dictionary=dictionary, coherence='u_mass')

print(coherence_model_cv.get_coherence(), coherence_model_umass.get_coherence())

0.653945115939642 -3.391984988178521


In [136]:
feature_names = vectorizer.get_feature_names_out()
n_top_words = 10
for topic_idx, topic_weights in enumerate(lsi_model.components_):
    top_indices = np.argsort(np.abs(topic_weights))[::-1][:10]
    top_terms = [terms[i] for i in top_indices]
    weights = [topic_weights[i] for i in top_indices]
    print(f"Topic #{topic_idx + 1}:")
    print(" + ".join(f"{weight:.3f}*\"{term}\"" for term, weight in zip(top_terms, weights)))

Topic #1:
0.355*"strategy" + 0.345*"economy" + 0.320*"markets" + 0.312*"finance" + 0.291*"trial" + 0.280*"business" + 0.271*"investing" + 0.262*"strategic" + 0.138*"inflation" + 0.116*"reuters"
Topic #2:
0.415*"news" + 0.348*"inflation" + 0.234*"yet" + 0.196*"second_quarter" + -0.160*"investing" + -0.160*"finance" + -0.156*"strategic" + -0.145*"trial" + -0.144*"strategy" + 0.144*"prime_minister"
Topic #3:
0.792*"news" + -0.340*"inflation" + 0.206*"announces" + -0.128*"prime_minister" + -0.113*"june" + -0.107*"second_quarter" + -0.107*"fed" + -0.104*"yet" + 0.098*"secretary" + -0.074*"open"


### Вивести документи, що описують кожну з тем

In [129]:
# Create dataframe
document_topics_df = pd.DataFrame(doc_topic_matrix, columns=[f"T{i+1}" for i in range(n_topics)])
document_topics_df.head(10)

,T1,T2,T3
0,0.393741,0.074718,-0.015287
1,0.131041,0.219543,-0.152422
2,0.257539,0.653417,-0.544072
3,0.406557,1.018732,-0.944707
4,0.998898,-0.385121,0.031455
5,0.104767,0.182960,-0.115881
6,0.089712,0.081335,-0.045964
7,0.178763,0.168972,-0.062208
8,0.062374,0.102356,-0.055187
9,0.097989,0.214122,-0.133571


In [130]:
# Топ 3 релевантних документів для кожної теми
for topic_idx in range(n_topics):
    print(f"\n=== Theme #{topic_idx + 1} ===")
    top_doc_indices = document_topics_df.iloc[:, topic_idx].abs().sort_values(ascending=False).head(3).index
    for i, doc_idx in enumerate(top_doc_indices):
        weight = document_topics_df.iloc[doc_idx, topic_idx]
        print(f"\nDocument {i + 1} (theme weight: {weight:.3f}):")
        print(data_df.loc[doc_idx, 'text'][:500], "...")  # перші 500 символів



=== Theme #1 ===

Document 1 (theme weight: 1.849):
$DOW $LYB - Cash In With 2 High-Yield Dividend Stocks: Passive-Income Stocks To Fight Inflation.  https://t.co/IzKZyYNy1k #markets #trading #finance ...

Document 2 (theme weight: 1.496):
2 Attractive High Dividend Yield Stocks To Buy.  https://t.co/UbtSvdsOs9 #business #stocks #markets ...

Document 3 (theme weight: 1.480):
#Growth #stocks outpace value stocks $SPY $QQQ $DIA $SPX $COMPQ $DJIA $XLY $XLK $XLC $XLI $XLRE $XLB  https://t.co/LzEmmlQJhI #economy #stockmarket ...

=== Theme #2 ===

Document 1 (theme weight: 1.924):
#Breaking: Inflation surges to new 40-year high as CPI rises 9.1% year over year in June, the highest increase since November 1981  https://t.co/PjGWGkKpdk by @alexandraandnyc  https://t.co/fyQt9joo5N ...

Document 2 (theme weight: 1.573):
BREAKING: UK inflation hits a new 40-year high in June, intensifying the cost of living crisis and adding pressure on the BOE to deliver an aggressive interest-rate hike next 

### Вивести найбільш важливі теми для випадково обраних чотирьох документів

In [135]:
random_docs = random.sample(range(len(data_df)), 4)
for idx in random_docs:
    print(f"\nDocument #{idx}")
    print(data_df.loc[idx, 'text'][:300], "...\n")
    top_topics = document_topics_df.iloc[idx].abs().sort_values(ascending=False).head(3)
    for topic_name, weight in top_topics.items():
        terms_str = topic_terms.get(topic_name, "(no terms)")
        print(f"- {topic_name} (weight: {weight:.3f})")


Document #4379
$CANO - Cano Health (CANO) Investor Presentation - Slideshow.  https://t.co/bivRsdKck3 #business #finance #investing ...

- T1 (weight: 0.896)
- T2 (weight: 0.423)
- T3 (weight: 0.127)

Document #5495
$AVDL: Avadel Pharmaceuticals receives tentative approval of LUMRYZ  https://t.co/MKdslugfxb ...

- T2 (weight: 0.018)
- T1 (weight: 0.008)
- T3 (weight: 0.008)

Document #701
ITALIAN HICP (YOY) (JUN) ACTUAL: 8.5% VS 8.5% PREVIOUS; EST 8.5% ...

- T3 (weight: 0.095)
- T2 (weight: 0.088)
- T1 (weight: 0.034)

Document #2432
The U.S. is on track to post the first annual deficit below $1 trillion since the start of the pandemic two and a half years ago.   https://t.co/3QqIKALkPi ...

- T2 (weight: 0.270)
- T1 (weight: 0.140)
- T3 (weight: 0.070)


### Використати текст carroll-alice.txt з корпусу gutenberg бібліотеки nltk та вивести ключові біграми.

In [48]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\mkriv\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [141]:
alice = gutenberg.sents('carroll-alice.txt')
alice_clean = [
    [re.sub(r"[^\w\s]", "", word.lower()) for word in sent if word.lower() not in stop_words and word not in string.punctuation]
    for sent in alice
]

In [142]:
# Діставання біграм за PMI
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(alice_clean)

# Фільтрація біграм, які зустрічаються рідко
finder.apply_freq_filter(10)

In [143]:
# Топ 10 біграм за PMI
top_bigrams = finder.nbest(bigram_measures.pmi, 10)
print("Top 10 key bigrams by PMI:")
for bigram in top_bigrams:
    print(f"{bigram[0]} {bigram[1]}")

Top 10 key bigrams by PMI:
march hare
mock turtle
white rabbit
minute two
oh dear
said caterpillar
poor alice
said king
thought alice
said cat
